In [15]:
import torch
import pandas as pd
# from torch.utils.data import DataLoader, SequentialSampler
from sklearn.preprocessing import MaxAbsScaler
import numpy as np
from sklearn.cluster import KMeans
from collections import Counter
from itertools import chain
from tqdm.auto import tqdm
import pickle
from pathlib import Path

from dataset import TwitterDataset
from autoencoders import LinearAutoEncoder

In [16]:
from dataset import TwitterDataset

dset = TwitterDataset(
        'timestamp',
        'spam_idx.pkl',
        embedding_path='embeddings/all-MiniLM-L6-v2',
)

loading from timestamp:   0%|          | 0/221 [00:00<?, ?it/s]

loading embeddings..:   0%|          | 0/221 [00:00<?, ?it/s]

no spam here!
loaded dataset. took 54.94291590899229 ms. Got 18932577 items


In [17]:
from autoencoders import load_encoder

model = load_encoder('models/LinearAutoEncoder.pkl','linear')
for p in model.parameters():
    p.requires_grad = False
model = model.cuda()

In [18]:
from torch.utils.data import DataLoader

loader = DataLoader(dset,batch_size=4096)

In [19]:
reduced = []
for batch in tqdm(loader):
    inp = batch['embedding'].cuda()
    out = model.encoder(inp)
    reduced.append(out)
    # break
reduced = torch.vstack(reduced)
    # break


  0%|          | 0/4623 [00:00<?, ?it/s]

In [24]:
start = pd.to_datetime(dset.timestamp[dset.sorted_idx][0])

end = start + pd.Timedelta('30m')

In [28]:
scaled = MaxAbsScaler().fit_transform(reduced.cpu().numpy())

In [20]:
from kmeans_pytorch.__init__ import kmeans

cluster_ids_x, cluster_centers = kmeans(
    X=reduced, num_clusters=7, distance='euclidean', device=torch.device('cuda:0'), iter_limit =100
)

running k-means on cuda:0..


[running kmeans]: 2it [00:00, 13.41it/s, center_shift=10.945193, iteration=2, tol=0.000100]

device is :cuda:0
device is :cuda:0
device is :cuda:0


[running kmeans]: 5it [00:00, 13.63it/s, center_shift=2.404896, iteration=6, tol=0.000100] 

device is :cuda:0
device is :cuda:0
device is :cuda:0


...

device is :cuda:0
device is :cuda:0
device is :cuda:0
device is :cuda:0


[running kmeans]: 56it [00:03, 15.41it/s, center_shift=0.000520, iteration=57, tol=0.000100]

device is :cuda:0
device is :cuda:0
device is :cuda:0
device is :cuda:0


[running kmeans]: 59it [00:03, 14.89it/s, center_shift=0.000093, iteration=59, tol=0.000100]

device is :cuda:0


In [22]:
torch.save({'cluster_ids': cluster_ids_x, 'cluster_centers': cluster_centers},'cluster_results/')

tensor([3, 6, 3,  ..., 5, 5, 0])

In [ ]:
# cluster_path = sorted(Path('cluster_results/kmeans_7_30m/parsed.pkl').glob('*.pkl'))

parsed_data = torch.load('cluster_results/kmeans_7_30m/parsed.pkl')

In [ ]:
slice_results = torch.load('cluster_results/kmeans_7_30m/slices/137.pkl')

In [ ]:
[len(ss['distance']) for ss in slice_results]

[5681, 6913, 2570, 9097, 4685, 3857, 5985]

In [ ]:
# : break
list(chain(*[c['topics'][:10] for c in parsed_data[0]]))

['twitter',
 'join',
 'telegram',
 'check',
 'tweet',
 'maybe',
 'cryptocurrencies',
 'follow',
 'bnb',
 'gives',
 '$',
 'nft',
 'token',
 'crypto',
 'join',
 'premium',
 'apesport',
 '+',
 'binance',
 'nfts',
 'roll',
 'bridge',
 'security',
 'like',
 'amp',
 'nt',
 'slow',
 'eat',
 'feet',
 'good',
 'testnet',
 'zk',
 'address',
 'claiming',
 'tokens',
 'v2',
 'evm',
 'compatible',
 'rollup',
 'learn',
 '$',
 'crypto',
 'binance',
 'btc',
 'bnb',
 'ftt',
 'ftx',
 'bitcoin',
 'project',
 'sol',
 'pump',
 'kucoin',
 'event',
 'signal',
 'big',
 'group',
 'happen',
 'wallstreetbets',
 'hours',
 '1',
 'security',
 'social',
 'amp',
 'people',
 'sushi',
 'volt',
 'roll',
 'medicare',
 'french',
 'republicans']